In [72]:
#Importar librerias
from sklearn.cross_validation import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.utils import column_or_1d

#Extraccion de la lista de ordenes de cabeceras seleccionadas
xlss = pd.read_excel('Process_considerations.xlsx', sheetname='FULL_DATASET_NC', 
                     header=None, index=False)
list_select = xlss.iloc[7,7].replace("c(","").replace(")","").split(",")

#En los casos de ordenes separados por ":" se toma los primeros
list_select = [_.split(":")[0] for _ in list_select]

#Extraccion de las cabeceras seleccionadas segun su numero indicado
matriz_head = xlss.iloc[:, range(2,6)].values
list_orden, list_head = [], []
for index in range(matriz_head.__len__()):
    if index % 2 == 0:
        list_orden.extend(matriz_head[index])
    else:
        list_head.extend(matriz_head[index])
del xlss
del matriz_head

#Limpio data vacia en la lista de cabeceras
list_head = [_.replace('"','') for _ in list_head if isinstance(_, str)]
list_head_select = [list_head[list_orden.index(_)] for _ in list_orden if str(_) in list_select]

#Importar Data Set 
dataset = pd.read_csv("datos_tc_uni.csv")
list_head.clear()
list_select.clear()

#Importar Data Set con las cabeceras dentro del csv
list_head_select = [_ for _ in list_head_select if _ in dataset.loc[0,:].to_dict().keys()]
print("Cabeceras seleccionadas dentro del CSV")
print(list_head_select)
print("//////////////////////////////////////////////////")
X = dataset.loc[:,list_head_select].values
list_head_select.clear()
y = dataset.loc[:,["TARGET_XF"]].values

#Limpiar Data de las columnas del csv
list_nan = ["", "NaN", "Nan", "None", "nan"]
def cleann(x):
    return x if str(x) not in list_nan else 0

cleann = np.vectorize(cleann)
X = cleann(X)

#Dividimos nuestra set de Training y Testing
X_train, X_test, y_train, y_test=train_test_split(X,y,test_size=0.25, random_state=0)

#Transformar a la misma escala
from sklearn.preprocessing import StandardScaler
sc_X=StandardScaler()
X_train = sc_X.fit_transform(X_train.astype(float))
X_test = sc_X.transform(X_test.astype(float))

#Ajustamos Regresion logistica para el Training set
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state=0)
classifier.fit(X_train, y_train.ravel())

#Exactitud:
print("Porcentaje de exactitud: ", classifier.score(X_test, y_test))

#Prediction del Test set
y_pred = classifier.predict(X_test)
#Construir la Matriz de Confusion
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print("Matriz de Confusion: ")
print(cm)




Cabeceras seleccionadas dentro del CSV
['LINEA_TC', 'INGRESO_BRUTO', 'EMP_REP_PP_SF', 'EMP_REP_CONV_SF', 'EMP_REP_VEH_SF', 'EMP_REP_HIP_SF', 'FLG_TC_SF', 'FLG_PP_SF', 'FLG_CONV_SF', 'FLG_VEH_SF', 'FLG_HIP_SF', 'FLG_BANCARIZADO', 'SALDO_TC_SF', 'IND_EMP_REP_PP_SF', 'IND_EMP_REP_CONV_SF', 'IND_EMP_REP_VEH_SF', 'IND_EMP_REP_HIP_SF', 'SALDO_TOT_ACT', 'EDAD', 'CAMP_TOT', 'RECENCIA_CAMP', 'CREC_ENT_REP_TC']
//////////////////////////////////////////////////
Porcentaje de exactitud:  0.762265153024
Matriz de Confusion: 
[[10241   803]
 [ 2778  1241]]
